In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import regionate as reg
import sectionate as sec

In [3]:
import numpy as np
import xgcm
import xarray as xr
import matplotlib.pyplot as plt

In [4]:
import pickle

## Load basin boundaries as an instance of `regionate.Regions` consisting of 35 instances of `regionate.Region`

In [5]:
import scipy.io
basins = scipy.io.loadmat('boundaries/PJ2010_abyssal_basin_boundaries.mat', simplify_cells=True, squeeze_me=True)['bsn_only']
region = reg.Regions({
    str(r['bsn_num'].replace("_", "")):
    reg.Region(
        str(r['bsn_num'].replace("_", "")),
        r['bsn'][0],
        r['bsn'][1]
    )
    for r in basins
})
    
# Slightly shift coordinates off of supergrid nodes to avoid rounding errors
for rname, r in region.regions.items():
    r.lons -= 1.e-5
    r.lats -= 1.e-5

## Conform regions to CM4p25 grid

In [6]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p25/c192_OM4_025_grid_No_mg_drag_v20160808_unpacked/ocean_hgrid.nc")
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p25_v7_allfixes/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    ocean_grid = ocean_grid.assign_coords({
        'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
        'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
        'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
        'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
        'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
        'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
        'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
        'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])
    })
    
    coords={
        'X': {'center': 'xh', 'outer': 'xq'},
        'Y': {'center': 'yh', 'outer': 'yq'},
    }
    grid = xgcm.Grid(ocean_grid, coords=coords, periodic=("X"))

In [7]:
region_gridded = reg.GriddedRegions({}, grid)

# Conform basin boundaries to model grid
print("Conforming regions to ocean grid: ", end="")
for (rname, r) in region.regions.items():
    print(rname, end=", ")
    region_gridded.regions[rname] = reg.GriddedRegion(
        rname,
        r.lons,
        r.lats,
        grid,
        positive_in=r.counterclockwise
    )

Conforming regions to ocean grid: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 

IndexError: index 1440 is out of bounds for axis 1 with size 1440

In [ ]:
reg.check_global_coverage(region_gridded)
region_gridded.find_all_overlaps(face_indices=True)

# `pickle` boundary indices and area mask
regionFile_CM4p25 = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region_gridded_CM4p25"
with open(regionFile_CM4p25, 'wb') as pickle_file:
    pickle.dump(region_gridded, pickle_file)

## Conform regions to CM4p125 grid

In [ ]:
# Load CMp125 grid
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    
    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc")
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    ocean_grid = ocean_grid.assign_coords({
        'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
        'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
        'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
        'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
        'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
        'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
        'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
        'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])
    })

    coords={
        'X': {'center': 'xh', 'outer': 'xq'},
        'Y': {'center': 'yh', 'outer': 'yq'},
    }
    grid = xgcm.Grid(ocean_grid, coords=coords, periodic=("X"))

In [ ]:
region_gridded = reg.GriddedRegions({}, ocean_grid, grid)

# Conform basin boundaries to model grid
print("Conforming regions to ocean grid: ", end="")
for (rname, r) in region.regions.items():
    print(rname, end=", ")
    region_gridded.regions[rname] = reg.GriddedRegion(
        rname,
        r.lons,
        r.lats,
        grid
    )

In [ ]:
# `pickle` boundary indices and area mask
regionFile_CM4p125 = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region_gridded_CM4p125"
with open(regionFile_CM4p125, 'wb') as pickle_file:
    pickle.dump(region_gridded, pickle_file)